## Lab: Mounting Blob Storage with Secrets API
---
This lab will show how to securely mount your storage locations within your Azure Databricks workspace. From the preceeding lecture, we know that we can use the Secrets API and the Databricks CLI to create, update, and manage secrets that can hold confidential information such as Access Keys, SAS tokens or Connection Strings. For Azure Blob Storage, we need to obfuscate the connection string as that is the only requried confidential data to mount it within the Databricks File System.

The first step will be to provision a Azure Blob Storage Account, if you have not already done so.  
  
If you have provivisioned your Azure Blob Storage Account, you know need to look up your SAS token key which should be under Settings -> Shared Access Signature. Click Generate SAS and connection string. Copy the SAS token.

Now open your command prompt and test to make sure you are connected to your workspace. To test type the following: `databricks workspace list`. If an error shows than you are not properly connected. Run through the steps in the Databricks CLI section of this workshop to connect your CLI to your Azure Databricks workspace.

The second command to run in your command prompt should be `databricks secrets -h`. This will show all the possible commands for the Secrets API. Look through this and familiarize yourself with the options.

We want to use the `create-scope` command first, to create a Secret Scope for our blob credentials. Let's check it out. Run `databricks secrets create-scope -h` in your command prompt.

It looks like there is one required field, `--scope [name of the scope]`. Let's create a scope with the name **blobsecret**. Run `databricks secrets create-scope --scope blobsecret` in your command prompt.  
  
To make sure it was created run `databricks secrets list-scopes`. It should be a row in the table that is outputted by this command.

Now it is time to create the secret itself. Going back to the list of commands under `databricks secrets`, we saw many command, but to create a secret we will need to use the `put` command. Run `databricks secrets put -h` to see what this command does and what is required to run it.  
  
As you can see there are **two** required parameters for this command: the scope for the secret and the name you wish to have for the secret itself. For this demo we will be using the scope we just created **blobsecret** and we will be using the name **blobkey**. Run `databricks secrets put --scope blobsecret --key blobkey` in your command prompt.

**IMPORTANT**: An editor (vim or Notepad) should now be displayed. There will be some text saying "Do not write below this line," follow that instruction. Copy and paste the SAS token Key value that you saved from the Azure Portal into the editor above the area where it says not to edit.   
  
**If `vi` or `vim` opens**
1. you will need to press the `i` key on your keyboard
2. `SHIFT + INS` where INS is the Insert Key at the top of your keyboard 
3. Then press `Esc`
4. type `:wq`

**If Notepad opens**
1. Copy and paste the value above the place where it says not to edit
2. `CTRL + S` to save
3. Close Notepad  
  
Now run the command `databricks secrets list --scope blobsecret` to make sure it was created.

Now we have one more secret to create, and this one is for the endpoint value.  
  
**Note**: The endpoint for Azure Blob Storage follows this format **wasbs://[your-container-name]@[your-storage-account-name].blob.core.windows.net**. 
  
You will need to grab your container name and your Storage Account name to form the endpoint. Once you have done this run the following command with **blobsecret** as the scope and **blobendpoint** as the Azure Blob Storage endpoint: `databricks secrets put --scope blobsecret --key blobendpoint`.

After copying the value into the editor, saving, and closing the editor run `databricks secrets list --scope blobsecret` to see the secrets you have created. There should be two from this demo.

If the above operations were successful we now can mount our blob storage using the Secrets here in the notebook. We need to use `dbutils.secrets` to make use of the secrets here in the notebook. Let's take a look at the commands available.

In [10]:
dbutils.secrets.help()

We will use the `dbutils.secrets.get()` command to get the secrets we need. In the cell below we grab the secrets that represent the confidential information to connect to our storage account. For the first parameter, the scope, for both we use **blobsecret**, and for the first variable we define in the following cell we use **blobendpoint** and the next variable **blobkey** for the second parameters respectively. Look below.

In [12]:
# the first piece of information described above goes into blobEndpoint
blobEndpoint = dbutils.secrets.get("blobsecret","blobendpoint")
# the second piece of information described above goes into blobKey
blobKey = dbutils.secrets.get("blobsecret","blobkey")

In [13]:
dbutils.fs.mount(
  source = blobEndpoint,
  mount_point = "/mnt/blobmountsecret",
  extra_configs = {"fs.azure.sas.<container-name>.<storage-account-name>.blob.core.windows.net":blobKey})

In [14]:
dbutils.fs.ls("/mnt/blobmountsecret")

As you can see, we have properly, securely mounted Azure Blob Storage into our Databricks workspace using the Databricks CLI and the Secrets API.